<a href="https://colab.research.google.com/github/RealBJr/sign-language-classifier/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print ('hello world')

!git commit -


hello world
fatal: not a git repository (or any of the parent directories): .git
